# Wrangling Countries & UN Regions
## By: Scott Kustes

### Objective:
Wrangle UN regions and subregions and associated countries for insertion into website database.

#### Dataset:
The original dataset was downloaded here: https://unstats.un.org/unsd/methodology/m49/

#### Contents
- <a href='#gather'>Data Gathering</a>
- <a href='#assess1'>Assess, Part 1</a>
- <a href='#clean1'>Clean, Part 1</a>
- <a href='#assess2'>Assess, Part 2</a>
- <a href='#clean2'>Clean, Part 2</a>
- <a href='#extract-un'>UN Regional Hierarchy Extraction</a>
- <a href='#assess3'>Assess, Part 3</a>
- <a href='#clean3'>Clean, Part 3</a>
- <a href='#extract-groups'>UN Groupings Extraction</a>
- <a href='#assess4'>Assess, Part 4</a>
- <a href='#clean4'>Clean, Part 4</a>
- <a href='#final'>Finished Dataframes</a>
- <a href='#db'>Insert into Database</a>

In [1]:
# Import necessary packages
import pandas as pd

<a id='gather'></a>
## Gather

In [2]:
countries = pd.read_csv( 'countries.csv' )
countries.sample(5)

Global Code Global Name  Region Code Region Name  Sub-region Code  \
230            1       World          9.0     Oceania             61.0   
42             1       World         19.0    Americas            419.0   
201            1       World        150.0      Europe             39.0   
146            1       World        150.0      Europe             39.0   
45             1       World         19.0    Americas            419.0   

                     Sub-region Name  Intermediate Region Code  \
230                        Polynesia                       NaN   
42   Latin America and the Caribbean                      29.0   
201                  Southern Europe                       NaN   
146                  Southern Europe                       NaN   
45   Latin America and the Caribbean                       5.0   

    Intermediate Region Name     Common Name         Official Name  \
230                      NaN          Tuvalu        Ellice Islands   
42                 Caribbean  Cayman Islands        Cayman Islands   
201                      NaN        Slovenia  Republic of Slovenia   
146                      NaN      Montenegro            Montenegro   
45             South America           Chile     Republic of Chile   

         Capital  M49 Code ISO-alpha3 Code Least Developed Countries (LDC)  \
230     Funafuti       798             TUV                               x   
42   George Town       136             CYM                             NaN   
201    Ljubljana       705             SVN                             NaN   
146    Podgorica       499             MNE                             NaN   
45      Santiago       152             CHL                             NaN   

    Land Locked Developing Countries (LLDC)  \
230                                     NaN   
42                                      NaN   
201                                     NaN   
146                                     NaN   
45                                      NaN   

    Small Island Developing States (SIDS) Developed / Developing Countries  
230                                     x                       Developing  
42                                    NaN                       Developing  
201                                   NaN                        Developed  
146                                   NaN                        Developed  
45                                    NaN                       Developing

<a id='assess1'></a>
## Assess, Part 1

In [3]:
countries['Global Code'].unique()

array([1], dtype=int64)

In [4]:
countries['Global Name'].unique()

array(['World'], dtype=object)

### Issues Found:
1) Drop columns: `Global Code` and `Global Name` - only 1 unique value

2) Rename columns: replace spaces with underscores, replace uppercase with lowercase

<a id='clean1'></a>
## Clean, Part 1
### 1) Drop columns `Global Code` and `Global Name`

Drop `Global Code` and `Global Name` columns due to each having only 1 unique value (1 and World, respectively).

#### Code

In [5]:
countries.drop( columns=['Global Code','Global Name'], axis=1, inplace=True )

#### Test

In [6]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
Region Code                                248 non-null float64
Region Name                                248 non-null object
Sub-region Code                            248 non-null float64
Sub-region Name                            248 non-null object
Intermediate Region Code                   108 non-null float64
Intermediate Region Name                   108 non-null object
Common Name                                249 non-null object
Official Name                              249 non-null object
Capital                                    243 non-null object
M49 Code                                   249 non-null int64
ISO-alpha3 Code                            248 non-null object
Least Developed Countries (LDC)            47 non-null object
Land Locked Developing Countries (LLDC)    32 non-null object
Small Island Developing States (SIDS)      53 non-null object
Developed / De

### 2) Rename Columns
Replace spaces with underscores, replace uppercase letters with lowercase

#### Code

In [7]:
# Dictionary of new column names
column_names = {
    'Region Code': 'region_code',
    'Region Name': 'region_name',
    'Sub-region Code': 'subregion_code',
    'Sub-region Name': 'subregion_name',
    'Intermediate Region Code': 'intermediate_region_code',
    'Intermediate Region Name': 'intermediate_region_name',
    'Common Name': 'country_common_name',
    'Official Name': 'country_official_name',
    'Capital': 'capital',
    'Territory of': 'territory_of',
    'M49 Code': 'un_m49',
    'ISO-alpha3 Code': 'iso_alpha3',
    'Least Developed Countries (LDC)': 'least_developed_countries',
    'Land Locked Developing Countries (LLDC)': 'landlocked_developing_countries',
    'Small Island Developing States (SIDS)': 'small_island_developing_states',
    'Developed / Developing Countries': 'developed_developing_countries'
}

countries.rename( mapper=column_names, axis=1, inplace=True )

#### Test

In [8]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
region_code                        248 non-null float64
region_name                        248 non-null object
subregion_code                     248 non-null float64
subregion_name                     248 non-null object
intermediate_region_code           108 non-null float64
intermediate_region_name           108 non-null object
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          47 non-null object
landlocked_developing_countries    32 non-null object
small_island_developing_states     53 non-null object
developed_developing_countries     248 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 29.3+ KB


<a id='assess2'></a>
## Assess, Part 2

In [9]:
countries['region_code'].unique()

array([  2., 142., 150.,   9.,  19.,  nan])

In [10]:
countries['region_name'].unique()

array(['Africa', 'Asia', 'Europe', 'Oceania', 'Americas', nan],
      dtype=object)

In [11]:
countries[ countries['region_code'].isnull() ]

region_code region_name  subregion_code subregion_name  \
9          NaN         NaN             NaN            NaN   

   intermediate_region_code intermediate_region_name country_common_name  \
9                       NaN                      NaN          Antarctica   

  country_official_name capital  un_m49 iso_alpha3 least_developed_countries  \
9            Antarctica     NaN      10        ATA                       NaN   

  landlocked_developing_countries small_island_developing_states  \
9                             NaN                            NaN   

  developed_developing_countries  
9                            NaN

In [12]:
countries['developed_developing_countries'].unique()

array(['Developing', 'Developed', nan], dtype=object)

In [13]:
countries['intermediate_region_name'].unique()

array(['Western Africa', nan, 'Middle Africa', 'Caribbean',
       'South America', 'Central America', 'Southern Africa',
       'Eastern Africa', 'Channel Islands'], dtype=object)

In [14]:
countries['region_name'].unique()

array(['Africa', 'Asia', 'Europe', 'Oceania', 'Americas', nan],
      dtype=object)

In [15]:
countries['subregion_name'].unique()

array(['Sub-Saharan Africa', 'Southern Asia', 'Northern Europe',
       'Southern Europe', 'Northern Africa', 'Polynesia',
       'Latin America and the Caribbean', nan, 'Western Asia',
       'Australia and New Zealand', 'Western Europe', 'Eastern Europe',
       'Northern America', 'South-eastern Asia', 'Eastern Asia',
       'Melanesia', 'Micronesia', 'Central Asia'], dtype=object)

### Issues Found:
1) Replace NaN with 0 in `region_code`, `subregion_code`, and `intermediate_region_code`

2) Replace NaN with 0 and 'x' with 1 in `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

3) Set datatypes:
- <strong>str:</strong> `region_code`, `subregion_code`, `intermediate_region_code`, padded with zeroes to three characters</li>
- <strong>bool:</strong> `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`

4) Set Antarctica's `region_code`, `subregion_code`, and `intermediate_region_code` to 999.

<a id='clean2'></a>
## Clean, Part 2
### 1) Fix Values in Columns with Region Codes
Replace NaN with 0 in region_code, subregion_code, and intermediate_region_code

#### Code

In [16]:
countries['region_code'].fillna( 0, inplace=True )
countries['subregion_code'].fillna( 0, inplace=True )
countries['intermediate_region_code'].fillna( 0, inplace=True )

#### Test

In [17]:
countries['region_code'].unique()

array([  2., 142., 150.,   9.,  19.,   0.])

In [18]:
countries['subregion_code'].unique()

array([202.,  34., 154.,  39.,  15.,  61., 419.,   0., 145.,  53., 155.,
       151.,  21.,  35.,  30.,  54.,  57., 143.])

In [19]:
countries['intermediate_region_code'].unique()

array([ 11.,   0.,  17.,  29.,   5.,  13.,  18.,  14., 830.])

### 2) Fix Values in 'Other Groupings'
Replace NaN with 0 and 'x' with 1 in `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

#### Code

In [20]:
countries['least_developed_countries'].fillna( 0, inplace=True )
countries['least_developed_countries'].replace( 'x', 1, inplace=True )

countries['landlocked_developing_countries'].fillna( 0, inplace=True )
countries['landlocked_developing_countries'].replace( 'x', 1, inplace=True )

countries['small_island_developing_states'].fillna( 0, inplace=True )
countries['small_island_developing_states'].replace( 'x', 1, inplace=True )

#### Test

In [21]:
countries['least_developed_countries'].unique()

array([0, 1], dtype=int64)

In [22]:
countries['landlocked_developing_countries'].unique()

array([0, 1], dtype=int64)

In [23]:
countries['small_island_developing_states'].unique()

array([0, 1], dtype=int64)

### 3) Set Datatypes
- <strong>str:</strong> `region_code`, `subregion_code`, `intermediate_region_code`, padded with zeroes to 3 characters
- <strong>bool:</strong> `least_developed_countries`, `landlocked_developing_countries`, and `small_island_developing_states`

#### Code

In [24]:
# Set str columns, convert to int64 first to remove decimal and following zero
countries['region_code'] = countries['region_code'].astype( 'int64' ).astype( 'str' )
countries['subregion_code'] = countries['subregion_code'].astype( 'int64' ).astype( 'str' )
countries['intermediate_region_code'] = countries['intermediate_region_code'].astype( 'int64' ).astype( 'str' )

# Pad with zeroes to three characters
countries['region_code'] = countries['region_code'].apply( lambda x: x.zfill(3) )
countries['subregion_code'] = countries['subregion_code'].apply( lambda x: x.zfill(3) )
countries['intermediate_region_code'] = countries['intermediate_region_code'].apply( lambda x: x.zfill(3) )

In [25]:
# Set bool columns
countries['least_developed_countries'] = countries['least_developed_countries'].astype( 'bool' )
countries['landlocked_developing_countries'] = countries['landlocked_developing_countries'].astype( 'bool' )
countries['small_island_developing_states'] = countries['small_island_developing_states'].astype( 'bool' )

#### Test

In [26]:
countries['region_code'].unique()

array(['002', '142', '150', '009', '019', '000'], dtype=object)

In [27]:
countries['subregion_code'].unique()

array(['202', '034', '154', '039', '015', '061', '419', '000', '145',
       '053', '155', '151', '021', '035', '030', '054', '057', '143'],
      dtype=object)

In [28]:
countries['intermediate_region_code'].unique()

array(['011', '000', '017', '029', '005', '013', '018', '014', '830'],
      dtype=object)

In [29]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
region_code                        249 non-null object
region_name                        248 non-null object
subregion_code                     249 non-null object
subregion_name                     248 non-null object
intermediate_region_code           249 non-null object
intermediate_region_name           108 non-null object
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
dtypes: bool(3), int64(1), object(11)
memory usage: 24.2+ KB


### 4) Set Antarctica's Region, Subregion, and Intermediate Region Codes to 999

#### Code

In [30]:
countries.loc[9,'region_code'] = '999'
countries.loc[9,'subregion_code'] = '999'
countries.loc[9,'intermediate_region_code'] = '999'

#### Test

In [31]:
countries.query( 'country_common_name == "Antarctica"' )

region_code region_name subregion_code subregion_name  \
9         999         NaN            999            NaN   

  intermediate_region_code intermediate_region_name country_common_name  \
9                      999                      NaN          Antarctica   

  country_official_name capital  un_m49 iso_alpha3  least_developed_countries  \
9            Antarctica     NaN      10        ATA                      False   

   landlocked_developing_countries  small_island_developing_states  \
9                            False                           False   

  developed_developing_countries  
9                            NaN

<a id='extract-un'></a>
## UN Regional Hierarchy Extraction
Extract UN Regional hierarchy and rationalize for insertion into website database.

### Create UN Region Hierarchy
In the UN Geoscheme, a region is made up of 1 or more subregions and a subregion is made up of 0 or more intermediate regions. Countries/areas are assigned at either the subregional or intermediate regional level (with Antarctica as the sole exception, having no regional assignment). Hierarchy levels are as follows: `Region -> Subregion -> Intermediate Region`

Antarctica is assigned to `region_code` 999.

The `countries` dataframe contains redundant information across the region, subregion, and intermediate region columns. Extract these values to create a hierarchy of UN regions in a new dataframe called `regions`. This new dataframe will contain 3 columns: 
- `region_code`: the value from the `region_code`, `subregion_code`, or `intermediate_region_code` column
- `region_name`: the value from the `region_name`, `subregion_name`, or `intermediate_region_name` column
- `parent_region_code`: the `region_code` for the region or subregion one level above in the hierarchy

In the `countries` dataframe, create a new column called `un_region` to hold the subregion or intermediate region to which each country is assigned.

#### Code

In [32]:
# Create an empty dataframe for holding the regions
regions = pd.DataFrame( columns=['region_code','region_name','parent_region_code'] )

# Iterrate through the countries to extract the UN region information
for row in countries.itertuples():
    # Get the region information from this row
    region_code = row.region_code
    region_name = row.region_name if region_code != '999' else 'Antarctica' # Antarctica is the sole exception so make it a top-level region
    subregion_code = row.subregion_code
    subregion_name = row.subregion_name
    intermediate_region_code = row.intermediate_region_code
    intermediate_region_name = row.intermediate_region_name
    
    # If region doesn't exist in regions dataframe already, add it
    if region_code not in regions['region_code'].unique():
        regions = regions.append( { 'region_code': region_code, 
                                    'region_name': region_name, 
                                    'parent_region_code': '000' }, 
                                    ignore_index=True )
    
    # If subregion doesn't exist in regions dataframe already, add it
    if ( subregion_code != '000' ) & ( subregion_code not in regions['region_code'].unique() ):
        regions = regions.append( { 'region_code': subregion_code, 
                                    'region_name': subregion_name, 
                                    'parent_region_code': region_code }, 
                                    ignore_index=True )

    # If intermediate region doesn't exist in regions dataframe already, add it
    if ( intermediate_region_code != '000' ) & ( intermediate_region_code not in regions['region_code'].unique() ):
        regions = regions.append( { 'region_code': intermediate_region_code, 
                                    'region_name': intermediate_region_name, 
                                    'parent_region_code': subregion_code }, 
                                    ignore_index=True )

In [33]:
# Function to populate un_region field in countries dataframe
# If intermediate_region_code is not 000, return the intermediate_region_code
# Else return subregion_code 
# Note that one entry contains 000 in subregion_code: Antarctica
# Antarctica was previously assigned a region_code of 000 so returning subregion_code of 000 will not cause problems
def get_un_region( row ):
    # If intermediate_region_code is not 000, return it
    if row['intermediate_region_code'] != '000':
        return row['intermediate_region_code']
    else:
        return row['subregion_code']

countries['un_region'] = countries.apply( get_un_region, axis=1 )

#### Test

In [34]:
regions.sort_values( 'parent_region_code' )

region_code                      region_name parent_region_code
0          002                           Africa                000
12         019                         Americas                000
9          009                          Oceania                000
5          150                           Europe                000
15         999                       Antarctica                000
3          142                             Asia                000
8          015                  Northern Africa                002
1          202               Sub-Saharan Africa                002
28         057                       Micronesia                009
27         054                        Melanesia                009
10         061                        Polynesia                009
18         053        Australia and New Zealand                009
13         419  Latin America and the Caribbean                019
22         021                 Northern America                019
26         030                     Eastern Asia                142
25         035               South-eastern Asia                142
4          034                    Southern Asia                142
17         145                     Western Asia                142
30         143                     Central Asia                142
19         155                   Western Europe                150
20         151                   Eastern Europe                150
7          039                  Southern Europe                150
6          154                  Northern Europe                150
29         830                  Channel Islands                154
23         018                  Southern Africa                202
24         014                   Eastern Africa                202
11         017                    Middle Africa                202
2          011                   Western Africa                202
21         013                  Central America                419
14         029                        Caribbean                419
16         005                    South America                419

In [35]:
countries[['subregion_code','intermediate_region_code','un_region']].sample(10)

subregion_code intermediate_region_code un_region
85             202                      011       011
89             419                      029       029
76             154                      000       154
6              039                      000       039
234            154                      000       154
130            039                      000       039
81             202                      017       017
104            035                      000       035
227            145                      000       145
201            039                      000       039

<a id='assess3'></a>
## Assess, Part 3

In [36]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 16 columns):
region_code                        249 non-null object
region_name                        248 non-null object
subregion_code                     249 non-null object
subregion_name                     248 non-null object
intermediate_region_code           249 non-null object
intermediate_region_name           108 non-null object
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
dtypes: bool(3), int64

In [37]:
countries[ countries['iso_alpha3'].isnull() ]

region_code region_name subregion_code   subregion_name  \
192         150      Europe            154  Northern Europe   

    intermediate_region_code intermediate_region_name country_common_name  \
192                      830          Channel Islands                Sark   

    country_official_name capital  un_m49 iso_alpha3  \
192                  Sark     NaN     680        NaN   

     least_developed_countries  landlocked_developing_countries  \
192                      False                            False   

     small_island_developing_states developed_developing_countries un_region  
192                           False                      Developed       830

In [38]:
countries[ countries['developed_developing_countries'].isnull() ]

region_code region_name subregion_code subregion_name  \
9         999         NaN            999            NaN   

  intermediate_region_code intermediate_region_name country_common_name  \
9                      999                      NaN          Antarctica   

  country_official_name capital  un_m49 iso_alpha3  least_developed_countries  \
9            Antarctica     NaN      10        ATA                      False   

   landlocked_developing_countries  small_island_developing_states  \
9                            False                           False   

  developed_developing_countries un_region  
9                            NaN       999

### Issues Found:
#### Tidiness
1) Drop 6 UN region columns: `region_code`,`region_name`,`subregion_code`,`subregion_name`,`intermediate_region_code`,`intermediate_region_name`

2) Create two columns, `developed` and `developing`, from `developed_developing_countries` column. 

3) Set datatype of `developed` and `developing` to bool.

4) Drop `developed_developing_countries`.

#### Quality
5) Replace NaN in `iso_alpha3` with 'NA'. The island of Sark has no ISO Alpha3 code.

<a id='clean3'></a>
## Clean, Part 3

### 1) Drop 6 UN region columns
Drop `region_code`,`region_name`,`subregion_code`,`subregion_name`,`intermediate_region_code`, and `intermediate_region_name`. These columns are no longer needed. This information is now contained in the `regions` dataframe to which each country is associated through the `un_region` column.

#### Code

In [39]:
countries.drop( columns=['region_code','region_name','subregion_code','subregion_name','intermediate_region_code','intermediate_region_name'], axis=1, inplace=True )

#### Test

In [40]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 10 columns):
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
dtypes: bool(3), int64(1), object(6)
memory usage: 14.5+ KB


### 2) Create `developed` and `developing` Columns

`developed` will contain 1 if `developed_developing_countries` contains 'Developed', 0 otherwise<br/>
`developing` will contain 1 if `developed_developing_countries` contains 'Developing', 0 otherwise

#### Code

In [41]:
countries['developed'] = countries['developed_developing_countries'].apply( lambda x: 1 if x == 'Developed' else 0 )
countries['developing'] = countries['developed_developing_countries'].apply( lambda x: 1 if x == 'Developing' else 0 )

#### Test

In [42]:
countries[['developed','developing','developed_developing_countries']].sample(5)

developed  developing developed_developing_countries
163          1           0                      Developed
8            0           1                     Developing
177          0           1                     Developing
20           1           0                      Developed
98           1           0                      Developed

### 3) Set datatype for `developed` and `developing` Columns to bool

#### Code

In [43]:
countries['developed'] = countries['developed'].astype('bool')
countries['developing'] = countries['developing'].astype('bool')

#### Test

In [44]:
countries['developed'].value_counts()

False    183
True      66
Name: developed, dtype: int64

In [45]:
countries['developing'].value_counts()

True     182
False     67
Name: developing, dtype: int64

In [46]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 12 columns):
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
developed_developing_countries     248 non-null object
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), int64(1), object(6)
memory usage: 15.0+ KB


### 4) Drop `developed_developing_countries` Column

#### Code

In [47]:
countries.drop( columns=['developed_developing_countries'], inplace=True )

#### Test

In [48]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         248 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), int64(1), object(5)
memory usage: 13.0+ KB


### 5) Replace NaN in `iso_alpha3` with 'NA'

The island of Sark has no ISO Alpha3 code.

#### Code

In [49]:
countries['iso_alpha3'].fillna( 'NA', inplace=True )

#### Test

In [50]:
countries[ countries['iso_alpha3'].isnull() ]

Empty DataFrame
Columns: [country_common_name, country_official_name, capital, un_m49, iso_alpha3, least_developed_countries, landlocked_developing_countries, small_island_developing_states, un_region, developed, developing]
Index: []

<a id='extract-groups'></a>
## UN Groupings Extraction
Extract and rationalize information in the five "Other Groupings" columns: `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`, `developed`, and `developing`.  This information will be stored in 2 database tables. 

The first table will contain five grouping titles: 'Least Developed Countries', 'Landlocked Developing Countries', 'Small Island Developing States', 'Developed', and 'Developing'.

The second table will contain mappings of country ID to grouping ID.

#### Code

In [51]:
# Create a dataframe for the Other Groupings
group_names = ['Least Developed Countries', 'Landlocked Developing Countries', 'Small Island Developing States', 'Developed Countries', 'Developing Countries']
un_groupings = pd.DataFrame( data=group_names, columns=['polity_group_name'] )

# Set the Polity Group Type to 1, which is the ID for "UN Stats Grouping" in polity_group_types table
# At the time of this analysis, this is the only entry in polity_group_types table
un_groupings['polity_group_type'] = 1

In [52]:
# Get the indexes for the countries in each grouping
least_developed = countries.query( 'least_developed_countries == True' ).index.to_list()
landlocked = countries.query( 'landlocked_developing_countries == True' ).index.to_list()
small_island = countries.query( 'small_island_developing_states == True' ).index.to_list()
developed = countries.query( 'developed == True' ).index.to_list()
developing = countries.query( 'developing == True' ).index.to_list()

In [53]:
# Create a blank dataframe to contain the country to grouping mappings
country_to_group = pd.DataFrame( columns=['country_id', 'group_id'] )

groups = ['least_developed','landlocked','small_island','developed','developing']
group_id = 0
for variable_name in groups:
    for value in eval( variable_name ):
        country_to_group = country_to_group.append( { 'country_id': value,
                                                      'group_id': group_id },
                                                      ignore_index=True )
    group_id += 1

#### Test

In [54]:
un_groupings

polity_group_name  polity_group_type
0        Least Developed Countries                  1
1  Landlocked Developing Countries                  1
2   Small Island Developing States                  1
3              Developed Countries                  1
4             Developing Countries                  1

In [55]:
country_to_group.sample(10)

country_id group_id
172        136        3
343        203        4
234         50        4
18          97        0
299        143        4
314        162        4
69         170        1
110        159        2
64         132        1
306        152        4

In [56]:
# Found this one by luck
country_to_group.query( 'country_id == 230' )

country_id group_id
42         230        0
129        230        2
365        230        4

In [57]:
# Is True in the correct columns?
countries.query( 'index == 230' )

country_common_name country_official_name   capital  un_m49 iso_alpha3  \
230              Tuvalu        Ellice Islands  Funafuti     798        TUV   

     least_developed_countries  landlocked_developing_countries  \
230                       True                            False   

     small_island_developing_states un_region  developed  developing  
230                            True       061      False        True

<a id='assess4'></a>
## Assess, Part 4

In [58]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
country_common_name                249 non-null object
country_official_name              249 non-null object
capital                            243 non-null object
un_m49                             249 non-null int64
iso_alpha3                         249 non-null object
least_developed_countries          249 non-null bool
landlocked_developing_countries    249 non-null bool
small_island_developing_states     249 non-null bool
un_region                          249 non-null object
developed                          249 non-null bool
developing                         249 non-null bool
dtypes: bool(5), int64(1), object(5)
memory usage: 13.0+ KB


### Issues Found:
#### Quality
1) Remove quotes from Common and Official Names. Several entries contain " in official name (ex: "Bonaire, Sint Eustatius, and Saba") due to commas in field breaking CSV formatting on input. Remove quotes.

#### Tidiness
2) Drop 5 UN grouping columns: `least_developed_countries`,`landlocked_developing_countries`,`small_island_developing_states`,`developed`,`developing`

<a id='clean4'></a>
## Clean, Part 4

### 1) Remove quotes from Official Names

Several entries contain " in official name (ex: "Bonaire, Sint Eustatius, and Saba") due to commas in field breaking CSV formatting on input. Remove quotes.

Also, the official name for Sri Lanka contains an odd space character that breaks on read. Fix it.

#### Code

In [59]:
countries[ countries['country_official_name'].str.contains('"') ]

country_common_name  \
0   "Saint Helena, Ascension, and Tristan da Cunha"   
25                                      BES Islands   

                              country_official_name     capital  un_m49  \
0   "Saint Helena, Ascension, and Tristan da Cunha"   Jamestown     654   
25              "Bonaire, Sint Eustatius, and Saba"  Kralendijk     535   

   iso_alpha3  least_developed_countries  landlocked_developing_countries  \
0         SHN                      False                            False   
25        BES                      False                            False   

    small_island_developing_states un_region  developed  developing  
0                            False       011      False        True  
25                            True       029      False        True

In [60]:
countries.loc[0,'country_official_name'] = "Saint Helena, Ascension, and Tristan da Cunha"
countries.loc[0,'country_common_name'] = "Saint Helena, Ascension, and Tristan da Cunha"
countries.loc[25,'country_official_name'] = "Bonaire, Sint Eustatius, and Saba"
countries.loc[209,'country_official_name'] = "Democratic Socialist Republic of Sri Lanka"

#### Test

In [61]:
countries.query( '( country_common_name == "BES Islands" ) | ( country_common_name == "Saint Helena, Ascension, and Tristan da Cunha") | ( country_common_name == "Sri Lanka" )' )

country_common_name  \
0    Saint Helena, Ascension, and Tristan da Cunha   
25                                     BES Islands   
209                                      Sri Lanka   

                             country_official_name                    capital  \
0    Saint Helena, Ascension, and Tristan da Cunha                  Jamestown   
25               Bonaire, Sint Eustatius, and Saba                 Kralendijk   
209     Democratic Socialist Republic of Sri Lanka  Sri Jayawardenepura Kotte   

     un_m49 iso_alpha3  least_developed_countries  \
0       654        SHN                      False   
25      535        BES                      False   
209     144        LKA                      False   

     landlocked_developing_countries  small_island_developing_states  \
0                              False                           False   
25                             False                            True   
209                            False                           False   

    un_region  developed  developing  
0         011      False        True  
25        029      False        True  
209       034      False        True

### 2) Drop 5 UN grouping columns
Drop `least_developed_countries`, `landlocked_developing_countries`, `small_island_developing_states`, `developed`, and `developing`. These columns are no longer needed. This information is now contained in the `un_groupings` and `country_to_group` dataframes.

#### Code

In [62]:
countries.drop( columns=['least_developed_countries','landlocked_developing_countries','small_island_developing_states','developed','developing'], axis=1, inplace=True )

#### Test

In [63]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 6 columns):
country_common_name      249 non-null object
country_official_name    249 non-null object
capital                  243 non-null object
un_m49                   249 non-null int64
iso_alpha3               249 non-null object
un_region                249 non-null object
dtypes: int64(1), object(5)
memory usage: 11.8+ KB


<a id='final'></a>
## Finished Dataframes

These dataframes are ready for database insertion.

In [64]:
# Sort by country_common_name, then output to csv
countries.sort_values( 'country_common_name', inplace=True )
countries.to_csv( 'countries_clean.csv', encoding='utf-16', index=False )
countries.sample(5)

country_common_name                    country_official_name  \
74                Faroe Islands                            Faroe Islands   
182                      Rwanda                       Republic of Rwanda   
60                      Denmark                       Kingdom of Denmark   
33       British Virgin Islands                   British Virgin Islands   
98   Heard and McDonald Islands  Territory of Heard and McDonald Islands   

        capital  un_m49 iso_alpha3 un_region  
74     Tórshavn     234        FRO       154  
182      Kigali     646        RWA       014  
60   Copenhagen     208        DNK       154  
33    Road Town      92        VGB       029  
98          NaN     334        HMD       053

In [65]:
# Sort by parent_region_code, then output to csv
regions.sort_values( 'parent_region_code', inplace=True )
regions.to_csv( 'un_regions.csv', index=False )
regions.sample(10)

region_code         region_name parent_region_code
15         999          Antarctica                000
11         017       Middle Africa                202
25         035  South-eastern Asia                142
26         030        Eastern Asia                142
20         151      Eastern Europe                150
27         054           Melanesia                009
4          034       Southern Asia                142
5          150              Europe                000
30         143        Central Asia                142
16         005       South America                419

In [66]:
un_groupings.to_csv( 'groupings.csv', index=False )
un_groupings

polity_group_name  polity_group_type
0        Least Developed Countries                  1
1  Landlocked Developing Countries                  1
2   Small Island Developing States                  1
3              Developed Countries                  1
4             Developing Countries                  1

In [67]:
country_to_group.to_csv( 'country_to_group.csv', index=False )
country_to_group.sample(10)

country_id group_id
143         47        3
220         32        4
258         82        4
354        218        4
177        163        3
126        221        2
127        224        2
81          10        2
294        138        4
359        224        4

<a id='db'></a>
# Insert Into Database

In [68]:
# Import classes and functions needed for this analysis from config module
# These are only available on my computer
from config import dbaccess, build_insert_query, un_region, polity_group, polity

# Create an instance of the DBAccess class for running queries
db = dbaccess.DBAccess()

## UN Regions
### Code

In [69]:
# Validate UN Regions prior to insert
validate_regions = un_region.UNRegion.validate_regions( regions )
if validate_regions:
    for error in validate_regions:
        print( error )
else:
    print( 'No errors' )

No errors


In [70]:
# Truncate table each run to prevent collisions
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'un_m49_regions' )

# Insert UN Regions
print( 'Expecting to insert:', regions.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_un_regions = db.run_query( build_insert_query('un_m49_regions', regions) )

# Output the results
if insert_un_regions['error'] == 0:
    print( insert_un_regions['rows'], 'records inserted' )
else:
    print( 'Error code:', insert_un_regions['error_code'] )
    print( 'Error message:', insert_un_regions['error_message'] )
    print( 'SQL State:', insert_un_regions['sqlstate'] )

Expecting to insert: 31 rows
31 records inserted


### Test

In [71]:
un_regions = db.run_query( 'SELECT * FROM un_m49_regions' )

# Output the results
if un_regions['error'] == 0:
    regions_df = pd.DataFrame.from_dict( un_regions['data'] )
    print( regions_df.sample(5) )
else:
    print( 'Error code:', un_regions['error_code'] )
    print( 'Error message:', un_regions['error_message'] )
    print( 'SQL State:', un_regions['sqlstate'] )

   region_code         region_name parent_region_code
26         017       Middle Africa                202
0          002              Africa                000
28         013     Central America                419
15         035  South-eastern Asia                142
29         029           Caribbean                419


## UN Groupings
### Code

In [72]:
# Get the IDs of all Polity Group Types from the database
valid_group_types = db.run_query( 'SELECT polity_group_type_id FROM polity_group_types' )

# Output the results
if valid_group_types['error'] == 0:
    group_types_df = pd.DataFrame.from_dict( valid_group_types['data'] )['polity_group_type_id'].values
    print( group_types_df )
else:
    print( 'Error code:', valid_group_types['error_code'] )
    print( 'Error message:', valid_group_types['error_message'] )
    print( 'SQL State:', valid_group_types['sqlstate'] )

[1]


In [73]:
# Validate Polity Groups prior to insert
validate_groups = polity_group.PolityGroup.validate_polity_groups( un_groupings, group_types_df )
if validate_groups:
    for error in validate_groups:
        print( error )
else:
    print( 'No errors' )

No errors


In [74]:
# Truncate table each run to prevent collisions
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'polity_groups' )

# Insert Polity Groups
print( 'Expecting to insert:', un_groupings.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_groupings = db.run_query( build_insert_query('polity_groups', un_groupings) )

# Output the results
if insert_groupings['error'] == 0:
    print( insert_groupings['rows'], 'records inserted' )
else:
    print( 'Error code:', insert_groupings['error_code'] )
    print( 'Error message:', insert_groupings['error_message'] )
    print( 'SQL State:', insert_groupings['sqlstate'] )

Expecting to insert: 5 rows
5 records inserted


### Test

In [75]:
polity_groups = db.run_query( 'SELECT * FROM polity_groups' )

# Output the results
if polity_groups['error'] == 0:
    polity_groups_df = pd.DataFrame.from_dict( polity_groups['data'] )
    polity_groups_df.set_index( 'polity_group_id', inplace=True )
    print( polity_groups_df.sample(5) )
else:
    print( 'Error code:', polity_groups['error_code'] )
    print( 'Error message:', polity_groups['error_message'] )
    print( 'SQL State:', polity_groups['sqlstate'] )

                               polity_group_name  polity_group_type
polity_group_id                                                    
1                      Least Developed Countries                  1
4                            Developed Countries                  1
5                           Developing Countries                  1
3                 Small Island Developing States                  1
2                Landlocked Developing Countries                  1


## Countries (Polities)
Insert countries into `polities` table. 

### Code

In [76]:
# Get the IDs of all Polity Types from the database
valid_polity_types = db.run_query( 'SELECT polity_type_id FROM polity_types' )

# Output the results
if valid_polity_types['error'] == 0:
    polity_types_df = pd.DataFrame.from_dict( valid_polity_types['data'] )['polity_type_id'].values
    print( polity_types_df )
else:
    print( 'Error code:', valid_polity_types['error_code'] )
    print( 'Error message:', valid_polity_types['error_message'] )
    print( 'SQL State:', valid_polity_types['sqlstate'] )

[1]


In [73]:
# Validate Polity Groups prior to insert
validate_polities = polity.Polity.validate_polity( countries, polity_types_df )
if validate_polities:
    for error in validate_polities:
        print( error )
else:
    print( 'No errors' )

No errors


In [74]:
# Truncate table each run to prevent collisions
# REMOVE WHEN DONE WITH NOTEBOOK
db.truncate_table( 'polity_groups' )

# Insert Polity Groups
print( 'Expecting to insert:', un_groupings.shape[0], 'rows' ) # How many rows am I expecting to be inserted?
insert_groupings = db.run_query( build_insert_query('polity_groups', un_groupings) )

# Output the results
if insert_groupings['error'] == 0:
    print( insert_groupings['rows'], 'records inserted' )
else:
    print( 'Error code:', insert_groupings['error_code'] )
    print( 'Error message:', insert_groupings['error_message'] )
    print( 'SQL State:', insert_groupings['sqlstate'] )

Expecting to insert: 5 rows
5 records inserted


### Test